Thanks to 
https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-1-fd544fab149#.3ebu974t6

One bandit with n arms, requires no state since there is only one bandit, rewards are not delayed but they are stationary and stochastic.

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
bandits = [.2,0,-.2,-5]
num_bandits = len(bandits)
def pullBandit(bandit):
    result = np.random.randn(1)
    if result > bandit:
        return 1
    else:
        return -1

In [5]:
tf.reset_default_graph()

weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [10]:
total_episodes = 1000
total_reward = np.zeros(num_bandits)
e = .1

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)

        reward = pullBandit(bandits[action])

        _,resp,ww = sess.run([update, responsible_weight, weights],
                            feed_dict={reward_holder:[reward],
                                       action_holder:[action]})
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))
        i+=1
print("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 bandits: [-1.  0.  0.  0.]
Running reward for the 4 bandits: [ -1.  -1.   1.  44.]
Running reward for the 4 bandits: [ -1.  -1.   1.  92.]
Running reward for the 4 bandits: [  -3.   -4.    2.  136.]
Running reward for the 4 bandits: [  -3.   -3.    1.  184.]
Running reward for the 4 bandits: [  -3.   -3.    2.  233.]
Running reward for the 4 bandits: [  -4.   -2.    3.  280.]
Running reward for the 4 bandits: [  -3.   -4.    3.  323.]
Running reward for the 4 bandits: [  -2.   -5.    2.  368.]
Running reward for the 4 bandits: [  -2.   -5.    4.  414.]
Running reward for the 4 bandits: [  -1.   -6.    4.  462.]
Running reward for the 4 bandits: [   0.   -6.    3.  508.]
Running reward for the 4 bandits: [  -1.   -6.    1.  555.]
Running reward for the 4 bandits: [   0.   -8.    3.  600.]
Running reward for the 4 bandits: [   0.   -8.    4.  649.]
Running reward for the 4 bandits: [   1.   -9.    4.  695.]
Running reward for the 4 bandits: [   2.   -9.    4.  74